In [1]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
load_dotenv()

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study"

#### TEST 용 객체 정의

In [39]:
from enum import Enum
from typing import List, Dict, Union

class Profile:
    def __init__(self, id: str, name: str, location: str, gender: str):
        self.id = id
        self.name = name
        self.location = location
        self.gender = gender

    def to_dict(self) -> Dict:
        return {
            "id": self.id,
            "name": self.name,
            "location": self.location,
            "gender": self.gender
        }

class Profiles:
    profile_list: List[Dict] = []

    @classmethod
    def get_profiles(cls, **kwargs) -> List[Dict]:
        results = cls.profile_list
        for key, value in kwargs.items():
            results = [profile for profile in results if profile.get(key) == value]
        return results

    @classmethod
    def add_profile(cls, profile: Union[Dict, Profile]) -> str:
        if isinstance(profile, Profile):
            profile = profile.to_dict()
        cls.profile_list.append(profile)
        return "Profile added successfully."

    @classmethod
    def delete_profile(cls, profile: Union[Dict, Profile]) -> str:
        profile_id = profile.get('id') if isinstance(profile, Dict) else profile.id
        cls.profile_list = [p for p in cls.profile_list if p.get('id') != profile_id]
        return "Profile deleted successfully."

In [40]:
from typing import Dict
from langchain.agents import tool

@tool
def get_profiles(**kwargs) -> List[Dict]:
    """
    id, name, location, gender 4가지의 argument들을 and 조건으로
    신상정보(profile)을 검색하여 return한다.
    """
    profiles = Profiles.get_profiles(**kwargs)
    if profiles:
        return profiles  # Assuming we return the first matching profile
    return []

@tool
def add_profile(profile: Dict) -> str:
    """
    id, name, location, gender 4가지 정보를 받아 하나의
    신상정보(profile)을 등록한다.
    gender 는 남성인 경우 'male', 여성인 경우 'female' 값만 유효하다.
    """
    #return Profiles.add_profile(profile)
    return 'Thanks'

@tool
def delete_profile(id: str) -> str:
    """
    id 정보를 받아 해당 신상정보(profile)을 삭제한다.
    """
    return Profiles.delete_profile({'id':id})

tools = [get_profiles, add_profile, delete_profile]

In [46]:
Profiles.add_profile({'id': '1', 'name': '김형기', 'location': '서울', 'gender': 'male'})

'Profile added successfully.'

In [47]:
Profiles.profile_list

[{'id': '1', 'name': '김형기', 'location': '서울', 'gender': 'male'}]

In [14]:
# ChatOllama는 create_tool_calling_agent 단계에서 error
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="aya", temperature=0.0)

In [41]:
from langchain_openai import ChatOpenAI        
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    api_key="ollama",
    model="aya",
    base_url="http://localhost:11434/v1",
)

In [18]:
# TEST
r = llm.invoke("한국에 대해 짧게 알려줘")
r

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


AIMessage(content='한국은 동아시아에 위치한 국가로, 아름다운 자연과 풍부한 역사 및 문화 유산을 자랑합니다. 한국은 약 7,000여 개의 섬으로 구성된 반도이며, 주요 도시로는 서울, 부산, 인천,대구 등이 있습니다. 한국은 경제적으로 빠르게 성장한 국가로 알려져 있으며, 특히 기술 및 제조업 분야에서 세계적인 수준의 경쟁력을 갖추고 있습니다. 또한 한국은 풍부한 역사와 문화 유산을 가지고 있는데, 유명한 곳으로는 경복궁, 남산타워, N서울타워 등 다양한 명소가 있습니다. 한국은 또한 맛있는 음식과 활기 넘치는 대중문화로도 잘 알려져 있습니다.', response_metadata={'token_usage': {'completion_tokens': 141, 'prompt_tokens': 14, 'total_tokens': 155}, 'model_name': 'aya', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-29f5fe78-2914-4f35-9053-fedaa9071365-0', usage_metadata={'input_tokens': 14, 'output_tokens': 141, 'total_tokens': 155})

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


In [42]:
system_prompt = \
"""
너는 인사팀 담당자야.
human의 요청에 맞춰서 신상정보(profile)을 조회(get_profiles), 등록(add_profile), 삭제(delete_profile)를 수행하라.
"""

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [43]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [44]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [45]:
input = """
내 이름은 김형기, 사는 곳은 서울, 남자야. 내 정보를 등록해줘
"""
agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...

Invoking: `add_profile` with `{'id': '1', 'name': '김형기', 'location': '서울', 'gender': 'male'}`




ValidationError: 1 validation error for add_profileSchema
profile
  field required (type=value_error.missing)

In [27]:
Profiles.profile_list

[]